## Assignment-02, Probability Model A First Look: An Introduction of Language Model
####  Assignment
Review the course online programming code;

Review the main questions;

Using wikipedia corpus to build a language model.

## 1. Review the course online programming code.

In [ ]:
!pwd

In [ ]:
dataset = '/Users/quark30/Desktop/NLP.csv'

In [ ]:
import os
os.path.exists(dataset)

In [ ]:
import pandas as pd
dataframe = pd.read_csv(dataset,sep=';')
dataframe[:5]

In [ ]:
all_articles = dataframe['content'].tolist()

In [1]:
import re

In [ ]:
#过滤掉所有的非文字和数字
def token(string):
    return ' '.join(re.findall('[\w|\d]+',string))

In [ ]:
token('此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。\nMIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。\n当然，关于MIUI 9的确切信息，我们还是等待官方消息。\n')

In [ ]:
all_articles = [token(str(a)) for a in all_articles]

In [ ]:
text = ''
for a in all_articles:
    text+=a
print('length of text:{}'.format(len(text)))

In [ ]:
TEXT = text

In [ ]:
from functools import reduce
txt_from_reduce = reduce(lambda a1,a2:a1+a2,all_articles[:10])

In [ ]:
import jieba
def cut(string): return list(jieba.cut(string))
ALL_TOKENS = cut(TEXT)
len(ALL_TOKENS)

In [ ]:
valida_tokens = [t for t in ALL_TOKENS if t.strip() and t != '\n'] #删空格
len(valida_tokens)

In [ ]:
#valida_tokens_1 = [t for t in ALL_TOKENS if not re.findall(' +', t)]

In [ ]:
valida_tokens[:10]

#### get frequency of words

In [ ]:
from collections import Counter
Counter([1,2,3,4,1,2,3,2,3])

In [ ]:
word_count = Counter(valida_tokens)

In [ ]:
frequencies = [f for w,f in word_count.most_common(100)]
x = [i for i in range(len(frequencies))]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
plt.plot(x, np.log(frequencies))

In [ ]:
frequencies_all = [f for w,f in word_count.most_common()]
frequencies_sum = sum(frequencies_all)

#### 如何解决没见过的单词？
假设全部的单词是已知的单词数目在加上现有单词的种类个数

In [ ]:
def laplace_smooth(counter,c=1):
    N = sum(counter.values())
    Nplus = N + c*(len(counter)+1)
    
    return lambda word : (counter[word]+c)/Nplus    

In [ ]:
laplace_prob = laplace_smooth(word_count)

In [ ]:
laplace_prob("你好")

In [ ]:
laplace_prob("我们")

我们来看一下我们所以单词中出现次数是1的单词：

In [ ]:
single_words = [w for w in word_count if word_count[w] == 1]
len(single_words)

这些单词的长度分布

In [ ]:
lengths = list(map(len,single_words))

In [ ]:
Counter(lengths).most_common()

参考 Good-Turing 的方法（但不是全部实现），我们使用出现次数是1的单词，重新评估了单词的概率，而且还考虑到了单词长度的影响。

其中的参数:

base: 值得是若单词长度过长，则概率衰减的速度 

prior: 指的是出现次数是0的单词比1的单词少的估计倍数，这个单词我们可以自己调节

In [ ]:
def simple_good_tuning_version(counter,base=1/26,prior = 1e-8):
    N = sum(counter.values())
    lengths = map(len,[w for w in counter if counter[w]==1])
    ones = Counter(lengths)
    longest = max(ones)
    
    def _get_prob(w):
        if w in counter:
            prob = counter[w]/N
        elif len(w) in ones:
            prob = prior*ones[len(w)]/N
        else:
            prob = prior*ones[longest]/N * base **(len(word)-longest)
        
        return prob
    return _get_prob
gt_prob = simple_good_tuning_version(word_count)

In [ ]:
gt_prob("我们")

In [ ]:
gt_prob("啊啊啊啊啊我")

In [ ]:
def get_prob(word):
    esp = 1/frequencies_sum
    if word in word_count:
        return word_count[word]/frequencies_sum
    else:
        return esp

In [ ]:
get_prob("我们")

In [ ]:
from functools import reduce

def product(numbers):
    return reduce(lambda n1,n2:n1*n2,numbers)
product([1,2,3,4,5])

In [ ]:
def language_model_one_gram(string):
    words = cut(string)
    return product([get_prob(w) for w in words])

In [ ]:
language_model_one_gram('广交会下个月举办')

In [ ]:
language_model_one_gram('一个掉在了民房上')

In [ ]:
sentences = """
这是一个比较正常的句子
这个一个比较罕见的句子
小明毕业于清华大学
小明毕业于秦华大学
""".split()

In [ ]:
for s in sentences:
    print(s,language_model_one_gram(s))

In [ ]:
need_compared = [
    "今天晚上请你吃大餐，我们一起吃日料 明天晚上请你吃大餐，我们一起吃苹果",
    "真事一只好看的小猫 真是一只好看的小猫",
    "我去吃火锅，今晚 今晚我去吃火锅"
]

for s in need_compared:
    s1, s2 = s.split()
    p1, p2 = language_model_one_gram(s1), language_model_one_gram(s2)
    
    better = s1 if p1 > p2 else s2
    
    print('{} is more possible'.format(better))
    print('-'*4 + ' {} with probility {}'.format(s1, p1))
    print('-'*4 + ' {} with probility {}'.format(s2, p2))
    

### 2-Gram
Get the combination probability of 2-grams

In [ ]:
valid_tokens = [str(t) for t in valida_tokens]

In [ ]:
all_2_gram_words = [''.join(valid_tokens[i:i+2]) for i in range(len(valid_tokens[:-2]))]

In [ ]:
_2_gram_sum = len(all_2_gram_words)
_2_gram_counter = Counter(all_2_gram_words)

def get_combination_prob(w1,w2):
    if w1+w2 in _2_gram_counter: return _2_gram_counter[w1+w2]/_2_gram_sum
    else:
        return 1/_2_gram_sum

In [ ]:
get_combination_prob('波音', '飞机')

In [ ]:
get_combination_prob('波音', '北京')

In [ ]:
def get_prob_2_gram(w1,w2):
    return get_combination_prob(w1,w2)/get_prob(w1)

In [ ]:
get_prob_2_gram('去', '北京')

In [ ]:
get_prob_2_gram('去', '沈阳')

In [ ]:
def language_model_of_2_gram(sentence):
    sentence_prob = 1
    words = cut(sentence)
    
    for i,word in enumerate(words):
        if i == 0:
            prob = get_prob(word)
        else:
            previous=words[i-1]
            prob = get_prob_2_gram(previous,word)
            sentence_prob*=prob
    return sentence_prob

In [ ]:
language_model_of_2_gram('小明今天抽奖抽到一台苹果手机')


In [ ]:
need_compared = [
    "今天晚上请你吃大餐，我们一起吃日料 明天晚上请你吃大餐，我们一起吃苹果",
    "真事一只好看的小猫 真是一只好看的小猫",
    "今晚我去吃火锅 今晚火锅去吃我",
    "洋葱奶昔来一杯 养乐多绿来一杯"
]

for s in need_compared:
    s1, s2 = s.split()
    p1, p2 = language_model_of_2_gram(s1), language_model_of_2_gram(s2)
    
    better = s1 if p1 > p2 else s2
    
    print('{} is more possible'.format(better))
    print('-'*4 + ' {} with probility {}'.format(s1, p1))
    print('-'*4 + ' {} with probility {}'.format(s2, p2))

## 2. Review the main points of this lesson.

## 3. Using Wikipedia dataset to finish the language model.


In [ ]:
'''
import wget
url = 'https://dumps.wikimedia.org/zhwiki/20190401/zhwiki-20190401-pages-articles-multistream1.xml-p1p162886.bz2'
wget.download(url)
use WikiExtractor to compilr all artucles into a list
'''

In [2]:
file_path = "/Users/quark30/Desktop/AA/wiki_00.txt"
import glob
#The 'glob' library contains a function, also called 'glob', 
# that finds files and directories whose names match a pattern. 
files = glob.glob(file_path)

In [3]:
all_articles_for_assign_3 = []
for name in files:
    try:
        text_file = open(name, "r", encoding="utf8")
        lines = text_file.readlines()
        all_articles_for_assign_3.append(lines)
    except IOError as exc: # not sure what error this is
        if exc.errno != errno.EISDIR:
            raise
all_articles_for_assign_3

[['<doc id="13" url="https://zh.wikipedia.org/wiki?curid=13" title="数学">\n',
  '数学\n',
  '\n',
  '数学是利用符号语言研究數量、结构、变化以及空间等概念的一門学科，从某种角度看屬於形式科學的一種。數學透過抽象化和邏輯推理的使用，由計數、計算、量度和對物體形狀及運動的觀察而產生。數學家們拓展這些概念，為了公式化新的猜想以及從選定的公理及定義中建立起嚴謹推導出的定理。\n',
  '\n',
  '基礎數學的知識與運用總是個人與團體生活中不可或缺的一環。對數學基本概念的完善，早在古埃及、美索不達米亞及古印度內的古代數學文本便可觀見，而在古希臘那裡有更為嚴謹的處理。從那時開始，數學的發展便持續不斷地小幅進展，至16世紀的文藝復興時期，因为新的科學發現和數學革新兩者的交互，致使數學的加速发展，直至今日。数学并成为許多國家及地區的教育範疇中的一部分。\n',
  '\n',
  '今日，數學使用在不同的領域中，包括科學、工程、醫學、經濟學和金融學等。數學對這些領域的應用通常被稱為應用數學，有時亦會激起新的數學發現，並導致全新學科的發展，例如物理学的实质性发展中建立的某些理论激发数学家对于某些问题的不同角度的思考。數學家也研究純數學，就是數學本身的实质性內容，而不以任何實際應用為目標。雖然許多研究以純數學開始，但其过程中也發現許多應用之处。\n',
  '\n',
  '西方语言中“數學”（）一詞源自於古希臘語的（），其有“學習”、“學問”、“科學”，以及另外還有個較狹義且技術性的意思－「數學研究」，即使在其語源內。其形容詞（），意思為"和學習有關的"或"用功的"，亦會被用來指"數學的"。其在英语中表面上的複數形式，及在法语中的表面複數形式\'，可溯至拉丁文的中性複數\'，由西塞罗譯自希臘文複數（），此一希臘語被亚里士多德拿來指「萬物皆數」的概念。\n',
  '\n',
  '汉字表示的「數學」一詞大約产生于中国宋元時期。多指象數之學，但有時也含有今天上的數學意義，例如，秦九韶的《數學九章》（《永樂大典》記，即《數書九章》也被宋代周密所著的《癸辛雜識》記爲《數學大略》）、《數學通軌》（明代柯尚遷著）、《数学钥》（清代杜知耕著）、《數學拾遺》（清代丁取忠撰）。直到1939年，經過中國

#### 数据清理 

In [ ]:
%pdb

In [ ]:
from IPython.core.debugger import set_trace

In [ ]:
len(all_articles_for_assign_3)

In [7]:
all_articles_for_assign_3 = all_articles_for_assign_3[0]

In [ ]:
len(all_articles_for_assign_3)

In [ ]:
'3111122233'.startswith(('1', '2'))

In [8]:
TEXT = ''
TEXT =[element.strip('\n') for element in all_articles_for_assign_3 if not (element.startswith(('<doc id', '</doc>')))]
# for element in all_articles_for_assign_3:
#     if element.startswith('<doc id') or element.startswith('</doc>'):
#         all_articles_for_assign_3.remove(element)
#     else:
#         TEXT += element
# TEXT.strip('\n')

In [29]:
### find_all_tokens
TEXT_TOKENS =''
for element in TEXT[:20]:
    sline = element.strip()
    if sline == ' ': continue
    TEXT_TOKENS += ' '.join(re.findall('[\w|\d]+', sline))
TEXT_TOKENS

'数学数学是利用符号语言研究數量 结构 变化以及空间等概念的一門学科 从某种角度看屬於形式科學的一種 數學透過抽象化和邏輯推理的使用 由計數 計算 量度和對物體形狀及運動的觀察而產生 數學家們拓展這些概念 為了公式化新的猜想以及從選定的公理及定義中建立起嚴謹推導出的定理基礎數學的知識與運用總是個人與團體生活中不可或缺的一環 對數學基本概念的完善 早在古埃及 美索不達米亞及古印度內的古代數學文本便可觀見 而在古希臘那裡有更為嚴謹的處理 從那時開始 數學的發展便持續不斷地小幅進展 至16世紀的文藝復興時期 因为新的科學發現和數學革新兩者的交互 致使數學的加速发展 直至今日 数学并成为許多國家及地區的教育範疇中的一部分今日 數學使用在不同的領域中 包括科學 工程 醫學 經濟學和金融學等 數學對這些領域的應用通常被稱為應用數學 有時亦會激起新的數學發現 並導致全新學科的發展 例如物理学的实质性发展中建立的某些理论激发数学家对于某些问题的不同角度的思考 數學家也研究純數學 就是數學本身的实质性內容 而不以任何實際應用為目標 雖然許多研究以純數學開始 但其过程中也發現許多應用之处西方语言中 數學 一詞源自於古希臘語的 其有 學習 學問 科學 以及另外還有個較狹義且技術性的意思 數學研究 即使在其語源內 其形容詞 意思為 和學習有關的 或 用功的 亦會被用來指 數學的 其在英语中表面上的複數形式 及在法语中的表面複數形式 可溯至拉丁文的中性複數 由西塞罗譯自希臘文複數 此一希臘語被亚里士多德拿來指 萬物皆數 的概念汉字表示的 數學 一詞大約产生于中国宋元時期 多指象數之學 但有時也含有今天上的數學意義 例如 秦九韶的 數學九章 永樂大典 記 即 數書九章 也被宋代周密所著的 癸辛雜識 記爲 數學大略 數學通軌 明代柯尚遷著 数学钥 清代杜知耕著 數學拾遺 清代丁取忠撰 直到1939年 經過中國數學名詞審查委員會研究 算學 與 數學 兩詞的使用狀況後 確認以 數學 表示今天意義上的數學含義數學有着久遠的歷史 它被認為起源於人類早期的生產活動 中國古代的六艺之一就有 數 數學一詞在西方有希腊语詞源 mathematikós 意思是 学问的基础 源于 máthema 科学 知识 学问史前的人類就已嘗試用自然的法則來衡量物質的多少 時間的長短等抽象的數量關係 比如时间单位有日 季節和年等

In [30]:
# --- change all characters into simplified chinese ('hanziconv' package)
from hanziconv import HanziConv
TEXT_TOKENS = HanziConv.toSimplified(TEXT_TOKENS)
TEXT_TOKENS.replace(' ','')

'数学数学是利用符号语言研究数量结构变化以及空间等概念的一门学科从某种角度看属于形式科学的一种数学透过抽象化和逻辑推理的使用由计数计算量度和对物体形状及运动的观察而产生数学家们拓展这些概念为了公式化新的猜想以及从选定的公理及定义中建立起严谨推导出的定理基础数学的知识与运用总是个人与团体生活中不可或缺的一环对数学基本概念的完善早在古埃及美索不达米亚及古印度内的古代数学文本便可观见而在古希腊那里有更为严谨的处理从那时开始数学的发展便持续不断地小幅进展至16世纪的文艺复兴时期因为新的科学发现和数学革新两者的交互致使数学的加速发展直至今日数学并成为许多国家及地区的教育范畴中的一部分今日数学使用在不同的领域中包括科学工程医学经济学和金融学等数学对这些领域的应用通常被称为应用数学有时亦会激起新的数学发现并导致全新学科的发展例如物理学的实质性发展中建立的某些理论激发数学家对于某些问题的不同角度的思考数学家也研究纯数学就是数学本身的实质性内容而不以任何实际应用为目标虽然许多研究以纯数学开始但其过程中也发现许多应用之处西方语言中数学一词源自于古希腊语的其有学习学问科学以及另外还有个较狭义且技术性的意思数学研究即使在其语源内其形容词意思为和学习有关的或用功的亦会被用来指数学的其在英语中表面上的复数形式及在法语中的表面复数形式可溯至拉丁文的中性复数由西塞罗译自希腊文复数此一希腊语被亚里士多德拿来指万物皆数的概念汉字表示的数学一词大约产生于中国宋元时期多指象数之学但有时也含有今天上的数学意义例如秦九韶的数学九章永乐大典记即数书九章也被宋代周密所着的癸辛杂识记爲数学大略数学通轨明代柯尚迁着数学钥清代杜知耕着数学拾遗清代丁取忠撰直到1939年经过中国数学名词审查委员会研究算学与数学两词的使用状况后确认以数学表示今天意义上的数学含义数学有着久远的历史它被认为起源于人类早期的生产活动中国古代的六艺之一就有数数学一词在西方有希腊语词源mathematikós意思是学问的基础源于máthema科学知识学问史前的人类就已尝试用自然的法则来衡量物质的多少时间的长短等抽象的数量关系比如时间单位有日季节和年等算术加减乘除也自然而然地产生了古代的石碑及泥版亦证实了当时已有几何的知识更进一步则需要写作或其他可记录数字的系统如符木或于印加帝国内用来储存数据的奇普历史上曾有过许多不同的记数系统在最初有历史记录的时候数

#### 分词操作

In [ ]:
import jieba
def cut(string): return list(jieba.cut(string))

ALL_TOKENS = cut(TEXT_TOKENS)

In [ ]:
# --- remove whitespaces 
valid_tokens = [t for t in ALL_TOKENS if t.strip()]

#### Get the frequences of word and implement the 1-gram model

In [ ]:
from collections import Counter
word_count = Counter(valid_token)

In [ ]:
# get frequences for all single words (1-grams)
frequences_all = [f for w, f in words_count.most_common()]
frequences_sum = sum(frequences_all)

In [ ]:
# now we define a procedure to calculate the probability for a give word
def get_prob(word):
    eps = 1 / frequences_sum # define a small constant, epsilon
    if word in words_count:
        return words_count[word] / frequences_sum
    else:
        return eps 

In [ ]:
# define a procedure to calculate the probability of a given sentence
# using the simplified 1-gram model
from functools import reduce

def product(numbers):
    "Return the multiplication product for a list of numbers"
    return reduce(lambda n1, n2: n1 * n2, numbers)

def sentence_prob_one_gram(string):
    "Return Pr(S) for a given sentence using the 1-gram model"
    words = cut(string)
    return product([get_prob(w) for w in words])


#### Use the 2-gram model

In [ ]:
all_2_grams_words = [''.join(valid_tokens[i:i+2]) for i in range(len(valid_tokens[:-2]))]
_2_gram_sum = len(all_2_grams_words)
_2_gram_counter = Counter(all_2_grams_words)

In [ ]:
# --- implement the 2-gram model
def get_comb_prob(w1,w2):
    "Return Pr(w1 * w2) if w1 + w2 occured in the text corpus"
    if w1+w2 in _2_gram_counter:
        return _2_gram_counter[w1+w2]/_2_gram_sum
    else:
        return 1/_2_gram_sum

def get_prob_two_gram(w1,w2):
    "Return Pr(w1 * w2) / Pr(w1) if w1 and w1 + w2 occured in the text corpus"
    return get_comb_prob(w1,w2)/get_prob(w1)
    
def sentence_prob_two_gram(sentence):
    "Return Pr(S) = Pr(w1) * [Pr(w1 * w2) / Pr(w1)] * ..."
    sentence_prob = 1
    words = cut(sentence)
    
    for i,word in enumerate(words):
        if i == 0: prob = get_prob(word)
        else: 
            previous = words[i-1]
            prob = get_prob_two_gram(previous,word)
        sentence_prob *= prob
    return sentence_prob

#### try a 3-gram model and more sentence pairs
Pr(S)=Pr(w1)* Pr(w1w2)/Pr(w1)* Pr(w1w2w3)/Pr(w2w3) * ... * Pr(wn-2wn-1wn)/Pr(wn-1wn)

In [ ]:
all_3_grams_words = [''.join(valid_tokens[i:i+3]) for i in range(len(valid_tokens[:-3]))]
_3_gram_sum = len(all_3_grams_words)
_3_gram_counter = Counter(all_3_grams_words)

In [ ]:
def get_three_comb_prob(w1,w2,w3):
    "Return Pr(w1 * w2 * w3) if w1+w2+w3 occured in the text corpus"
    if w1+w2+w3 in _3_gram_counter:
        return _3_gram_counter[w1+w2+w3]/_3_gram_sum
    else:
        return 1/_3_gram_sum
    
def get_prob_three_gram(w1,w2,w3):
    "Return Pr(w1 * w2 * w3) / Pr(w2 * w3) if w2+w3 and w1+w2+w3 occured in the text corpus"
    return get_three_comb_prob(w1,w2,w3)/get_comb_prob(w2,w3)

def sentence_prob_three_gram(sentence):
    "Return Pr(S) = Pr(w1) * [Pr(w1 * w2) / Pr(w1)] * [Pr(w1 * w2 * w3) / Pr(w2 * w3)] ..."
    sentence_prob = 1
    words = cut(sentence)
    
    for i, word in enumerate(words):
        if i == 0: 
            prob = get_prob(word)
        elif i == 1: 
            previous = words[i-1]
            prob = get_prob_two_gram(previous,word)
        else:
            word_n_2 = words[i-2]
            word_n_1 = words[i-1]
            prob = get_prob_three_gram(word_n_2,word_n_1,word)
        sentence_prob *= prob
    return sentence_prob

##### Compared to the previous learned parsing and pattern match problems. What's the advantage and disavantage of Probability Based Methods?

advantage: the probabilit-based model is simpler and more flexible than the pattern-match approach
disadvantage: the predicting power of the probability-based model is highly dependent on the text corpus we supply (or, the training data we use).